In [3]:
import os,getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
os.environ['SERPAPI_API_KEY'] = getpass.getpass('Serp API Key:')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass.getpass('Hugging FACE Hub API Token:')



## JekyllHyde - A Self Moderating System for Social Media
> Vectorize our Query Text , then Search for the vectors with closest distance 


In [4]:
from langchain import PromptTemplate
import numpy as np 

jekyll_template = """
You are a Social Media Post Commenter , you will repsond to the following post with a {sentiment} response.
Post:" {social_post}"
Comment:
"""

jekyll_prompt_template = PromptTemplate(
    input_variables = ["sentiment","social_post"],
    template = jekyll_template, 
)

random_sentiment = "nice"
if np.random.rand() < 0.3:
    random_sentiment = "mean"

social_post = "I can't believe I'm learning about LangChain in this Course , there is so much to learn and so far the instructions have been so helpful. Im having a lot of fun leaning! #AI"

jekyll_prompt = jekyll_prompt_template.format(
    sentiment = random_sentiment,
    social_post=social_post
)
print(f"JKelly Prompt:{jekyll_prompt}")

JKelly Prompt:
You are a Social Media Post Commenter , you will repsond to the following post with a nice response.
Post:" I can't believe I'm learning about LangChain in this Course , there is so much to learn and so far the instructions have been so helpful. Im having a lot of fun leaning! #AI"
Comment:



In [5]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from langchain.llms import OpenAI

jekyll_llm = OpenAI(model="text-babbage-001")

In [6]:
from langchain.chains import LLMChain
from better_profanity import profanity

jekyll_chain = LLMChain(
    llm=jekyll_llm,
    prompt=jekyll_prompt_template,
    output_key="jekyll_said",
    verbose=False,
)

jekyll_said = jekyll_chain.run(

    {"sentiment":random_sentiment, "social_post":social_post}
)

cleaned_jekyll_said = profanity.censor(jekyll_said)
print(f"Jekyll said:{cleaned_jekyll_said}")

Jekyll said:
Thanks for your feedback! We're happy you're enjoying the course and the instructions so far. Keep learning and we'll keep you updated on our progress.


### Hyde Moderator

In [7]:
hyde_template = """
You are hide the moderator of online forum, you are strict and will not tolerate any negative comments. 
You will look at this next comment from a user and , if it is at all negative, you will replace it with symbols and post that,
but if it seems nice , you will let it remain as is and repeat it word for word.
Original Comment: {jekyll_said}
Edited comment:
"""

hyde_prompt_template = PromptTemplate(
    input_variables=["jekyll_said"],
    template=hyde_template,
)

hyde_llm = OpenAI(model="text-davinci-003")

hyde_chain=LLMChain(
    llm=hyde_llm,prompt=hyde_prompt_template,verbose=False
)

hyde_says = hyde_chain.run({"jekyll_said":jekyll_said})
print(f"Hyde says:{hyde_says}")

Hyde says:Thanks for your feedback! We're happy you're enjoying the course and the instructions so far. Keep learning and we'll keep you updated on our progress.


In [8]:
from langchain.chains import SequentialChain

jekyllhyde_chain = SequentialChain(
    chains=[jekyll_chain,hyde_chain],
    input_variables=["sentiment","social_post"],
    verbose=True,
)

jekyllhyde_chain.run({"sentiment":random_sentiment, "social_post":social_post})



> Entering new SequentialChain chain...

> Finished chain.


"Great to hear you're enjoying the course!"